In [1]:
import random
import os
import numpy as np
import sklearn
import torch
from torch.cuda import manual_seed_all
import matplotlib as mpl
from matplotlib import pyplot as plt
import torch.nn as nn

PROJECT_ROOT_DIR = "."
CHAPTER_ID = "data"
AUDIO_PATH = os.path.join(PROJECT_ROOT_DIR, "audio", CHAPTER_ID)
os.makedirs(AUDIO_PATH, exist_ok=True)

In [ ]:
# DenseNet Model

class densenet:
    """ 
    DenseNet Class, derived from Pytorch. Intended for model manipulation (i.e. unfreezing layers, etc.)
    To use model, try (densenet).model(data)
    May change to reflect manual implementation of densenet161

    """
    def __init__(self):
        self.model = torchvision.models.densenet161()

    def layer_change(self):
        """
        Unfreeze layers of densenet model per specifications
        """
        pass
        

In [ ]:
# Define training and testing loops

def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    # Compute prediction and loss
    X = X.to(device)
    y = y.to(device)
    pred = model(X)
    global loss # for future retrieval
    loss = loss_fn(pred, y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 20 == 0:
        loss, current = loss.item(), (batch + 1) * len(X)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  global test_loss # for future retrieval
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      X = X.to(device)
      y = y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
# Model Testing
test_mod = densenet()


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 10
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(test_mod.model.parameters(), lr=0.0001)


for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    #Run training loop
    test_mod.model.train()
    train_loop(train_dataloader, test_mod.model, loss_fn, optimizer)
    test_mod.model.eval()

    # Test on Validation set, retrieve loss
    test_loop(val_dataloader, test_mod.model, loss_fn)